# Distribución posterior predictiva


## Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co

## Referencias

1. Alvaro Montenegro, [Curso de Estadística Bayesiana](https://github.com/AprendizajeProfundo/Estadistica-Bayesiana), 2020
2. Guangyuan Gao, "Bayesian Claims Reserving Methods in Non-life Insurance with Stan. An Introduction", Springer, 2018

# Introduction

En esta lección, revisamos el concepto de distribución predictiva. Esta distribución es la base de las herramientas para el chequeo de los modelos. 

Usaremos la distribución predictiva para predecir datos no observados, y para replicar datos observados.

# Revisión de la predictiva posterior



En el marco Bayesiano, idealmente queremos dividir los datos en un conjunto de entrenamiento y un conjunto de prueba y hacer la verificación predictiva posterior en el conjunto de datos de prueba.


Alternativamente, podemos elegir una estadística de prueba cuya distribución predictiva no dependa de parámetros desconocidos en el modelo, sino principalmente de la hipótesis que se verifica.


Entonces no hay necesidad de tener un conjunto de datos de prueba por separado. Sin embargo, cuando se usan los mismos datos para ajustar y verificar el modelo, esto debe llevarse a cabo con precaución, ya que el
el procedimiento puede ser conservador.



# Distribución predictiva


La distribución predictiva es la distribución de observaciones futuras dada la muestra actual.

Supongamos que $ y_ {n + 1} $ es una observación futura que es independiente de los datos observados $ \mathbf{y}$, dado el correspondiente parámetro $\boldsymbol{\theta}$. 


Entonces, la distribución predictiva para $ y_{n + 1} $ viene dada por

$$
\begin{equation*}
p(y_{n+1}|\mathbf{y}).
\end{equation*}
$$

Observe que

$$
\begin{align*}
p(y_{n+1}|\mathbf{y})&=\int
p(y_{n+1},\mathbf{\theta}|\mathbf{y})d\mathbf{\theta}\\
&= \int
f(y_{n+1}|\mathbf{\theta},\mathbf{y})p(\mathbf{\theta}|\mathbf{y})d\mathbf{\theta}\\
&=\int
f(y_{n+1}|\mathbf{\theta})p(\mathbf{\theta}|\mathbf{y})d\mathbf{\theta}
\end{align*}
$$

Es decir que

$$
\begin{equation*}
p(y_{n+1}|\mathbf{y}) = \int
f(y_{n+1}|\mathbf{\theta})p(\mathbf{\theta}|\mathbf{y})d\mathbf{\theta},
\end{equation*}
$$

y en consecuencia los valores predichos pueden ser calculados dentro del algorimo de muestreo.  Supongamos que en el paso $m$ después de convergencia se tiene que la muestra para $\boldsymbol{\theta}$ es $\boldsymbol{\theta}^{(m)}$. Entonces un valor predictivo puede ser obtenido a partir de la densidad posterior
predictiva

$$
\begin{equation*}
f(y|\mathbf{\theta}^{(m)})
\end{equation*}
$$

Dentro del algoritmo de muestreo se procede así: en el paso $m$

1. Se obtiene la muestra $\mathbf{\theta}^{(m)} \sim p(\mathbf{\theta}|\mathbf{y})$ (sección *model* de Stan).
2. Se obtiene el valor predicho $y_{n+1} \sim f(y|\mathbf{\theta}^{(m)})$ (sección *generated quantities* de Stan).

Observe que en el numeral 2, se está usando la función de verosimilitud (que es una densidad) con parámetro $\mathbf{\theta}^{(m)}$.

Cada observación $y_i$ en el caso de los modelos de regresión  depende no solamente del parámetro $\mathbf{\theta}$, sino también de un vector de variables regresoras o predictoras $\mathbf{x}_i$. En este caso tenemos que la posterior predictiva es dada por 

$$
\begin{equation*}
p(y_{n+1}|\mathbf{y},x_{n+1}) = \int
f(y_{n+1}|\mathbf{\theta},x_{n+1})p(\mathbf{\theta}|\mathbf{y})d\mathbf{\theta},
\end{equation*}
$$





# Predicción de datos no observados

En esta sección usamos el modelo de regresión Binomial para ilustrar la predicción de datos no observados.

## Ejemplo Modelo de Regresión Binomial

En la lección [Modelos Bayesianos de regresión](./Modelos_Bayesianos_Regresion.ipynb) introdujimos el modelo de regresión Binomial dado por

$$
\begin{align}
E[y_i] &= \mu_i= N_i\pi_i\\
\pi_i &=  \text{inv_logit}(\alpha +  \mathbf{x_i}^T\mathbf{\beta}) \\
\alpha &\sim \mathcal{N}(\mu_{\alpha},\sigma_{\alpha}^2)\\
\mathbf{\beta} &\sim \mathcal{N}(\mathbf{\mu}_{\beta},\rho_{\beta}^2\mathbf{I})\\
y_i &\sim \text{Binomial}(N_i,\pi_i), \hspace{3mm} i =1,\ldots,n.
\end{align}
$$

Aqui cada observación $y_i$ depende de $\pi_i$ y de su vector predictor $\mathbf{x}_i$. Supongamos que $\alpha^{(m)}$ y $\mathbf{\beta}^{(m)}$ son las muestras respectivamente de $\alpha$ y $\mathbf{\beta}$  en el paso $m$ del algortimo de muestreo. Entonces una muestra $y_*$ (no observada) en este paso dado el vector predictor $\mathbf{x}_*$, el cual no hace parte de los datos se obtiene como sigue. Sea $N_*$ el número de ensayos, y denotemos por $y_*^{(m)}$ la predicción de $y_*$  en el paso $m$. Entonces se tiene  que

$$
\begin{align}
{\pi}_*^{(m)} &=  \text{inv_logit}(\alpha^{(m)} +  \mathbf{x}_*^T{\mathbf{\beta}^{(m)}}) \\
y_*^{(m)} & \sim \text{Binomial}(N_*,{\pi}_*^{(m)}) 
\end{align}
$$

En Stan $\alpha$ y $\mathbf{\beta}$  son obtenidos en la sección *model* y los dos pasos anteriores para obtener la predicción son escritos en la sección *generated quantities*.

El siguiente fragmento de código ilustra como implementar el algoritmo anterior. La variable *y_pred* es utilizada para hacer la prediccón en cada paso.


In [2]:
# Magic STAN
import pystan
%load_ext stanmagic

In [16]:
%%stan -f binomial_reg.stan
//  Binomial Regression  Model
// Prediction of non-observed data

data{
int<lower=0> n; // número de datos
int<lower=0> p; // número de variables predictoras
int<lower=0> N[n]; // número de ensayos para cada observación
int<lower=0> y; // datos observados
matrix[n,p] X; // variables predictoras observadas
vector[n] x_p; // vector predictor no observado
int<lower=0> Np; // Número de ensayos para la predicción
}

parameters{
    real alpha;
    vector[p] beta;   
}

transformed parameters{
  vector<lower=0,upper=1>[n] pi; 
  pi = inv_logit(alpha + X * beta);
}


model{
  // priors
  alpha ~ normal(0,10);
  beta ~ normal(0,10);
  // likelihood   
  y ~ binomial_logit(N, pi);
}

generated quantities {
    real y_pred;
    real pp;
    pp = inv_logit(alpha + dot_product(x_p,beta));
    y_pred = binomial_rng(Np,pp);
}


Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [binomial_reg.stan]
_stan_model.model_name -> Name of stan model [binomial_reg_model]
_stan_model.model_code -> Model code [//  Binomial Regress ....]


In [ ]:
stacks_lasso = '''
data {
  int<lower=0> N;
  int<lower=0> p;
  real Y[N];
  matrix[N,p] x;
} 

// to standardize the x's 
transformed data {
  real z[N,p];
  real mean_x[p];
  real sd_x[p];
  for (j in 1:p) { 
    mean_x[j] <- mean(col(x,j)); 
    sd_x[j] <- sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] <- (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta0; 
  real beta[p]; 
  real<lower=0> sigmasq; 
  real<lower=0> phi;
} 

transformed parameters {
  real<lower=0> sigma;
  real mu[N];

  sigma <- sqrt(2) * sigmasq;
  for (n in 1:N)
    mu[n] <- beta0 + beta[1] * z[n, 1] + beta[2] * z[n, 2] + beta[3] * z[n, 3];
}

model {
  beta0 ~ normal(0, 316); 
  phi ~ gamma(0.01, 0.01);
  beta ~ normal(0, sqrt(phi));
  sigmasq ~ inv_gamma(.001, .001); 
  for (n in 1:N) 
    Y[n] ~ double_exponential(mu[n], sigmasq); 
} 

generated quantities {
  real b0;
  real b[p];
  real outlier_1;
  real outlier_3;
  real outlier_4;
  real outlier_21;

  for (j in 1:p)
    b[j] <- beta[j] / sd_x[j];
  b0 <- beta0 - b[1] * mean_x[1] - b[2] * mean_x[2] - b[3] * mean_x[3];

  outlier_1  <- step(fabs((Y[1] - mu[1]) / sigma) - 2.5);
  outlier_3  <- step(fabs((Y[3] - mu[3]) / sigma) - 2.5);
  outlier_4  <- step(fabs((Y[4] - mu[4]) / sigma) - 2.5);
  outlier_21 <- step(fabs((Y[21] - mu[21]) / sigma) - 2.5);
}
'''

# Producción de datos replicados

En los métodos de validación del modelo es común replicar los datos observados. Replicar un dato significa "predecir un dato observado". La diferencia con el caso anterior es que ahora para replicar el dato se utiliza el predictor de los datos observados.

De acuerdo con la notación de l a sección anterior, para replicar el dato $y_i$, se usará su predictor $\mathbf{x_i}$.

Para ilustar la replica de datos en el modelo de regresión binomial refactorizamos el código anterior. 

## Réplica de observaciones en el modelo de regresión Binomial

In [ ]:
# Magic STAN
import pystan
%load_ext stanmagic

In [19]:
%%stan -f binomial_reg_2.stan
//  Binomial Regression  Model
// Replication of observed data

data{
int<lower=0> n; // número de datos
int<lower=0> p; // número de variables predictoras
int<lower=0> N[n]; // número de ensayos para cada observación
int<lower=0> y; // datos observados
matrix[n,p] X; // variables predictoras observadas
}

parameters{
    real alpha;
    vector[p] beta;   
}

transformed parameters{
  vector<lower=0,upper=1>[n] pi; 
  pi = inv_logit(alpha + X * beta);
}


model{
  // priors
  alpha ~ normal(0,10);
  beta ~ normal(0,10);
  // likelihood   
  y ~ binomial_logit(N, pi);
 k ~ binomial (Np,)...
}

generated quantities {
    int y_rep[n];
    vector[n] pp;
    pp = inv_logit(alpha + X * beta);
    y_rep = binomial_rng(N,pp);
}


Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [binomial_reg_2.stan]
_stan_model.model_name -> Name of stan model [binomial_reg_2_model]
_stan_model.model_code -> Model code [//  Binomial Regress ....]


# Tarea

1. Consulte un conjunto de datos para una regresión Binomial. Consulte por ejemplo el texto del profesor Gilberto de Paula o en los ejemplos de Gamlss.
2. Modifique los códigos para los modelos de  la [lección de modelos Bayesianos de regresión](./Modelos_Bayesianos_Regresion.ipynb)

In [ ]:
import seaborn as sb

traces = fit_lasso.extract(permuted=True)